In [3]:
#!/usr/bin/python
import MySQLdb
import numpy as np  

# Numpy arrays to store things 
upc_arr = np.array([],dtype=int)
transactions = np.array([], dtype=int)

# connect to database and post any relevant errors
try:
    db = MySQLdb.connect(host="coffee-gave-me-gas.cgzqmhf3sjbn.us-east-2.rds.amazonaws.com",    # your host, usually localhost
                        user="root",         # your username
                        password="csc4112018",  # your password
                        db="projectcoffee")        # name of the data base
            
    # you must create a Cursor object. It will let
    #  you execute all the queries you need
    cur = db.cursor()

    # Use all the SQL you like
    cur.execute("SELECT productUPC FROM product")
    
    # print all the first cell of all the rows
    for productUPC in cur:
        upc = str(productUPC)[1:-2]
        upc_arr = np.append(upc_arr, upc)

    #db.close()
except Exception as e:
    print("Failed to connect to database...", "Error:", e)



ImportError: No module named MySQLdb

In [27]:
# What are the 20 top-selling products at each store? 
def query1():
    cur.execute("""select Rank, productName as Product, units as 'Units Sold', storeName as Store
from (select product_ID as upc, amountSold as units_sold, store_ID, storeName,
			-- Leveraged mysql session variables to track ranking 
			-- If the store_ID is = to the previous one than increase the rank, otherwise start back at 1
			  @cur_rank := IF(@store = store_ID, @cur_rank + 1, 1) as Rank, 
              @store := store_ID,
              @units := amountSold as units  
				-- In order to get the correct ranking order a Subquery of each stores products sales in descending order is needed
				from  (	select product_ID, amountSold, store_ID, storeName 					 
							from sales inner join store on store.ID = sales.store_ID
							order by store_ID, amountSold desc) as top_items
) as rankings inner join product on upc = product.ID
-- Cap the ranking at the top 20 items
where rank <= 20""")

    for rank, name, units, store in cur:
        n = name[1:-2]
    #     p = price[1:-2]
        r = rank
        r = int(r)
        print(r, n, units, store)

# What are the 20 top-selling products in each state?
def query2():
    cur.execute(""" select Rank, productName as Product, units as 'Units Sold', storeState as State
from (select product_ID as upc, amountSold as units_sold, store_ID, storeState,
			-- Leveraged mysql session variables to track ranking 
			-- If the storeStae is = to the previous one than increase the rank, otherwise start back at 1
			  @cur_rank := IF(@cur_state = storeState, @cur_rank + 1, 1) as Rank, 
              @cur_state := storeState,
              @units := amountSold as units  
				-- In order to get the correct ranking order a Subquery that orders by State and amount sold in descending order is needed
				from  (	select product_ID, amountSold, store_ID, storeState
							from sales inner join store on store.ID = sales.store_ID
							order by storeState, amountSold desc) as top_items
) as rankings inner join product on upc = product.ID
-- Cap the ranking at the top 20 items
where rank <= 20""")

    for rank, name,units, state in cur:
        n = name[1:-2]
        s = state
        r = rank
        r = int(r)
        print(r, n, units ,s)

# Which 5 stores have the most sales
def query3():
    cur.execute(""" select storeName as Store, sum(amountSold) as 'Total Sales'
from sales inner join store on store.ID = sales.store_ID
group by store_ID -- Allows the aggreate function sum based on each store to occur
order by sum(amountSold) desc -- Order the results from largest to smallest
limit 5	-- Return only the top 5
 """)

    for store, units in cur:
        print(store, units)

In [28]:
x = query3()

Hmart 24 97016
Hmart 38 69995
Hmart 42 65330
Hmart 22 64744
Hmart 65 59567


In [6]:
#save_prices(sale_amount)
db.close()